<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/DDQN_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.optimizers import Adam
import numpy as np
print(f'tf version: {tf.__version__}\nOk')

In [0]:
class DuelingDeepQNetwork(keras.Model):
    def __init__(self, n_actions, fc1_dims, fc2_dims):
        super(DuelingDeepQNetwork, self).__init__()
        self.dense1 = keras.layers.Dense(fc1_dims, activation='relu') 
        self.dense2 = keras.layers.Dense(fc2_dims, activation='relu') 
        self.V = keras.layers.Dense(1, activation=None)
        self.A = keras.layers.Dense(n_actions, activation=None)

    def call(self, state):
        x = self.dense1(state)
        x = self.dense2(x)
        V = self.V(x)
        A = self.A(x)
        Q = (V + (A - tf.math.reduce_mean(A, axis=1, keepdims=True)))
        
        return Q

    # may be unnecessary
    def advantage(self, state):
        x = self.dense1(state)
        x = self.dense2(x)
        A = self.A(x)

        return A


class ReplayBuffer():
    def __init__(self, max_size, input_shape):
        self.mem_size = max_size
        self.mem_cntr = 0
        self.state_memory = np.zeros((self.mem_size, *input_shape), 
                                     dtype=np.float32) 
        self.new_state_memory = np.zeros((self.mem_size, *input_shape),
                                         dtype=float32) 
        self.action_memory = np.zeros(self.mem_size, dtype=int32)
        self.reward_memory = np.zeros(self.mem_size, dtype=np.float32) 
        self.terminal_memory = np.zeros(self.mem_size, dtype=np.bool) 

    def store_transition(self, state, action, reward, state_, done):
        index = mem_cntr % mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        self.action_memory[index] = action
        self.reward_memory[index] = reward
        self.terminal_memory[index] = done

        mem_cntr += 1

    def sample_buffer(self, batch_size):
        max_mem = min(self.mem_cntr, self.mem_size)
        batch = np.random.choice(max_mem, batch_size, replace=False)

        states = self.state_memory[batch]
        new_states = self.new_state_memory[batch]
        actions = self.action_memory[batch]
        rewards = self.reward_memory[batch]
        dones = self.terminal_memory[batch]

        return states, new_states, actions, rewards, dones

class Agent():
    def __init__(self, lr, gamma, n_actions, epsilon, batch_size,
                 input_dims, epsilon_dec=1e-3, eps_end=0.01,
                 mem_size=1000000, fname='DDQN-Keras.h5', fc1_dims=128,
                 fc2_dims=128, replace=100):
        self.action_space = [i for i in range(n_actions)]
        # self.lr = lr
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_dec = epsilon_dec
        self.eps_end = eps_end
        self.fname = fname
        self.replace = replace
        
        self.learn_step_counter = 0
        self.memory = ReplayBuffer(max_size=mem_size, input_shape=input_dims)  
        self.q_eval = DuelingDeepQNetwork(n_actions, fc1_dims, fc2_dims)  
        self.q_next = DuelingDeepQNetwork(n_actions, fc1_dims, fc2_dims)  
        
        self.q_eval.compile(optimizer=Adam(learning_rate=lr), loss='mean_squared_error')
        self.q_next.compile(optimizer=Adam(learning_rate=lr), loss='mean_squared_error')

        
print(f'Ok')

In [0]:
ddqn = DuelingDeepQNetwork(n_actions=9, fc1_dims=16,fc2_dims=128)

# agent = Agent(lr=0.001, gamma=0.001, n_actions=9, epsilon=0.95, batch_size=2048, input_dims=1)
help(Agent)
Agent.__dict__